In [15]:
import os
import shutil

# Base directory
base_dir = 'Decoder-BS16-deep'

# Numbered folders to process
numbered_folders = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']

print("="*80)
print(f"Duplicating experiment folders in {base_dir}")
print("="*80)
print()

# Process each numbered folder
for folder_name in numbered_folders:
    folder_path = os.path.join(base_dir, folder_name)
    
    if not os.path.exists(folder_path):
        print(f"⚠ {folder_name}/ does not exist, skipping...")
        continue
    
    print(f"\nProcessing: {folder_name}/")
    print("-"*80)
    
    # Process both NPU and PIM folders
    for folder_type in ['npu', 'pim']:
        subfolder_path = os.path.join(folder_path, folder_type)
        
        if not os.path.exists(subfolder_path):
            print(f"  ⚠ {folder_name}/{folder_type}/ does not exist, skipping...")
            continue
        
        print(f"\n  {folder_name}/{folder_type}/:")
        
        # Get existing numbered folders (2-9)
        existing_folders = []
        for item in os.listdir(subfolder_path):
            item_path = os.path.join(subfolder_path, item)
            if os.path.isdir(item_path) and item.isdigit():
                num = int(item)
                existing_folders.append((num, item, item_path))
        
        if not existing_folders:
            print(f"    No numbered folders found")
            continue
        
        # Sort by number
        existing_folders.sort(key=lambda x: x[0])
        
        # Get the source folders (2-9)
        source_folders = [(num, name, path) for num, name, path in existing_folders if num >= 2 and num <= 9]
        
        if not source_folders:
            print(f"    No source folders (2-9) found to copy from")
            continue
        
        print(f"    Found {len(source_folders)} source folders: {[f[1] for f in source_folders]}")
        
        # Create folders 10-12 by cycling through source folders
        target_number = 10
        source_index = 0
        
        while target_number <= 12:
            # Check if this number already exists
            if any(num == target_number for num, _, _ in existing_folders):
                print(f"    Folder {target_number} already exists, skipping...")
                target_number += 1
                continue
            
            # Get source folder (cycle through 2-9)
            source_num, source_name, source_path = source_folders[source_index % len(source_folders)]
            
            # Create target path
            target_name = str(target_number)
            target_path = os.path.join(subfolder_path, target_name)
            
            # Copy the folder
            try:
                shutil.copytree(source_path, target_path)
                print(f"    ✓ Created {target_number} (copied from {source_num})")
            except Exception as e:
                print(f"    ✗ ERROR creating {target_number}: {e}")
            
            target_number += 1
            source_index += 1

print(f"\n{'='*80}")
print("✓ Folder duplication complete!")
print(f"{'='*80}")


Duplicating experiment folders in Decoder-BS16-deep


Processing: 1st/
--------------------------------------------------------------------------------

  1st/npu/:
    Found 8 source folders: ['2', '3', '4', '5', '6', '7', '8', '9']
    ✓ Created 10 (copied from 2)
    ✓ Created 11 (copied from 3)
    ✓ Created 12 (copied from 4)

  1st/pim/:
    Found 8 source folders: ['2', '3', '4', '5', '6', '7', '8', '9']
    ✓ Created 10 (copied from 2)
    ✓ Created 11 (copied from 3)
    ✓ Created 12 (copied from 4)

Processing: 2nd/
--------------------------------------------------------------------------------

  2nd/npu/:
    Found 8 source folders: ['2', '3', '4', '5', '6', '7', '8', '9']
    ✓ Created 10 (copied from 2)
    ✓ Created 11 (copied from 3)
    ✓ Created 12 (copied from 4)

  2nd/pim/:
    Found 8 source folders: ['2', '3', '4', '5', '6', '7', '8', '9']
    ✓ Created 10 (copied from 2)
    ✓ Created 11 (copied from 3)
    ✓ Created 12 (copied from 4)

Processing: 3rd/
-------

## Decoder-BS64 NPU Analysis - Process All Numbered Folders


In [13]:
import pandas as pd
import re
import os
import glob
from collections import defaultdict

# Base directory for Decoder-BS64
base_dir = 'Decoder-BS16'

# Numbered folders to process
numbered_folders = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']

print("="*80)
print(f"DECODER-BS64 NPU ANALYSIS")
print("="*80)
print()

# Process each numbered folder
for folder_name in numbered_folders:
    npu_folder = os.path.join(base_dir, folder_name, 'npu')
    
    if not os.path.exists(npu_folder):
        print(f"⚠ {npu_folder} does not exist, skipping...")
        continue
    
    print(f"\n{'='*80}")
    print(f"Processing: {folder_name}/npu/")
    print(f"{'='*80}\n")
    
    # Find all SA_stage_E.tsv files in this npu folder
    tsv_files = glob.glob(os.path.join(npu_folder, '**/SA_stage_E.tsv'), recursive=True)
    
    if not tsv_files:
        print(f"  No SA_stage_E.tsv files found in {npu_folder}")
        continue
    
    print(f"Found {len(tsv_files)} SA_stage_E.tsv file(s)\n")
    
    # Process each TSV file (each experiment folder 2-9)
    for input_file in sorted(tsv_files):
        # Extract experiment number from path
        exp_match = re.search(r'/(\d+)/SA_stage_E\.tsv', input_file)
        exp_num = exp_match.group(1) if exp_match else 'unknown'
        
        print(f"  Processing experiment {exp_num}...")
        
        # Read the TSV file
        df = pd.read_csv(input_file, sep='\t')
        
        # Extract expert operations - only TotalCycle
        expert_data = defaultdict(lambda: {'param_load': None, 'fc1': None, 'gelu': None, 'fc2': None})
        
        for idx, row in df.iterrows():
            op_name = row['OpName']
            
            # Check if it's an expert operation
            match = re.match(r'layer0\.ffn\.moe_expert\.(\d+)\.(param_load|fc1|fc2|gelu)', op_name)
            
            if match:
                expert_num = int(match.group(1))
                operation = match.group(2)
                expert_data[expert_num][operation] = row['TotalCycle']
        
        # Create DataFrame with only total cycles
        summary_data = []
        
        for expert_num in sorted(expert_data.keys()):
            param_load_raw = expert_data[expert_num]['param_load'] if expert_data[expert_num]['param_load'] else 0
            fc1 = expert_data[expert_num]['fc1'] if expert_data[expert_num]['fc1'] else 0
            gelu = expert_data[expert_num]['gelu'] if expert_data[expert_num]['gelu'] else 0
            fc2 = expert_data[expert_num]['fc2'] if expert_data[expert_num]['fc2'] else 0
            
            # Multiply param_load by 64
            param_load = param_load_raw * 64 if param_load_raw != 0 else 'N/A'
            
            # Calculate total compute (fc1 + gelu + fc2)
            total_compute = fc1 + gelu + fc2
            
            row_data = {
                'Expert Number': expert_num,
                'param_load': param_load,
                'fc1': fc1,
                'gelu': gelu,
                'fc2': fc2,
                'total compute': total_compute
            }
            summary_data.append(row_data)
        
        summary_df = pd.DataFrame(summary_data)
        
        # Save to text file with same name as input but .txt extension
        output_file = os.path.splitext(input_file)[0] + '.txt'
        with open(output_file, 'w') as f:
            f.write(f"NPU Expert Operations - Total Cycles\n")
            f.write(f"Decoder-BS64/{folder_name}/npu/{exp_num}\n")
            f.write("="*80 + "\n")
            f.write(summary_df.to_string(index=False))
        
        print(f"    ✓ Processed {len(expert_data)} experts → {output_file}")

print(f"\n{'='*80}")
print(f"✓ DECODER-BS64 NPU ANALYSIS COMPLETE!")
print(f"{'='*80}")
print()
print("Summary files (.txt) created for each experiment folder.")
print("Each file contains expert operation timings for all 64 experts.")


DECODER-BS64 NPU ANALYSIS


Processing: 1st/npu/

Found 8 SA_stage_E.tsv file(s)

  Processing experiment 2...
    ✓ Processed 51 experts → Decoder-BS16/1st/npu/2/SA_stage_E.txt
  Processing experiment 3...
    ✓ Processed 51 experts → Decoder-BS16/1st/npu/3/SA_stage_E.txt
  Processing experiment 4...
    ✓ Processed 56 experts → Decoder-BS16/1st/npu/4/SA_stage_E.txt
  Processing experiment 5...
    ✓ Processed 51 experts → Decoder-BS16/1st/npu/5/SA_stage_E.txt
  Processing experiment 6...
    ✓ Processed 52 experts → Decoder-BS16/1st/npu/6/SA_stage_E.txt
  Processing experiment 7...
    ✓ Processed 48 experts → Decoder-BS16/1st/npu/7/SA_stage_E.txt
  Processing experiment 8...
    ✓ Processed 53 experts → Decoder-BS16/1st/npu/8/SA_stage_E.txt
  Processing experiment 9...
    ✓ Processed 49 experts → Decoder-BS16/1st/npu/9/SA_stage_E.txt

Processing: 2nd/npu/

Found 8 SA_stage_E.tsv file(s)

  Processing experiment 2...
    ✓ Processed 51 experts → Decoder-BS16/2nd/npu/2/SA_stage_E.txt

## Decoder-BS64 PIM Analysis - Process All Numbered Folders


In [14]:
import pandas as pd
import re
import os
import glob
from collections import defaultdict

# Base directory for Decoder-BS64
base_dir = 'Decoder-BS16'

# Numbered folders to process
numbered_folders = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']

print("="*80)
print(f"DECODER-BS64 PIM ANALYSIS")
print("="*80)
print()

# Process each numbered folder
for folder_name in numbered_folders:
    pim_folder = os.path.join(base_dir, folder_name, 'pim')
    
    if not os.path.exists(pim_folder):
        print(f"⚠ {pim_folder} does not exist, skipping...")
        continue
    
    print(f"\n{'='*80}")
    print(f"Processing: {folder_name}/pim/")
    print(f"{'='*80}\n")
    
    # Find all SA_stage_E.tsv files in this pim folder
    tsv_files = glob.glob(os.path.join(pim_folder, '**/SA_stage_E.tsv'), recursive=True)
    
    if not tsv_files:
        print(f"  No SA_stage_E.tsv files found in {pim_folder}")
        continue
    
    print(f"Found {len(tsv_files)} SA_stage_E.tsv file(s)\n")
    
    # Process each TSV file (each experiment folder 2-9)
    for input_file in sorted(tsv_files):
        # Extract experiment number from path
        exp_match = re.search(r'/(\d+)/SA_stage_E\.tsv', input_file)
        exp_num = exp_match.group(1) if exp_match else 'unknown'
        
        print(f"  Processing experiment {exp_num}...")
        
        # Read the TSV file
        df = pd.read_csv(input_file, sep='\t')
        
        # Detect BS (batch size) from folder path to determine activation divider
        bs_match = re.search(r'BS(\d+)', input_file)
        if bs_match:
            bs_value = int(bs_match.group(1))
            # Determine divider based on batch size
            if bs_value == 64:
                activation_divider = 32
            elif bs_value == 32:
                activation_divider = 64
            elif bs_value == 16:
                activation_divider = 128
            else:
                activation_divider = 1  # Default if unknown BS
            divider_note = f"BS{bs_value} - Activation divider: {activation_divider}"
        else:
            activation_divider = 1
            divider_note = "No BS detected - Using default divider: 1"
        
        # Extract activation movement operations
        activation_movements = {}
        for idx, row in df.iterrows():
            op_name = row['OpName']
            if 'activation_movement' in op_name:
                activation_movements[op_name] = row['TotalCycle']
        
        # Calculate total activation cycles with divider (rounded to nearest integer)
        total_activation_cycles = 0
        activation_details = []
        for op_name, cycles in activation_movements.items():
            divided_cycles = round(cycles / activation_divider)
            activation_details.append(f"{op_name}: {divided_cycles} cycles")
            total_activation_cycles += divided_cycles
        
        # Extract expert operations - only TotalCycle (no param_load for PIM)
        expert_data = defaultdict(lambda: {'fc1': None, 'gelu': None, 'fc2': None})
        
        for idx, row in df.iterrows():
            op_name = row['OpName']
            
            # Check if it's an expert operation
            match = re.match(r'layer0\.ffn\.moe_expert\.(\d+)\.(fc1|fc2|gelu)', op_name)
            
            if match:
                expert_num = int(match.group(1))
                operation = match.group(2)
                expert_data[expert_num][operation] = row['TotalCycle']
        
        # Create DataFrame with only total cycles
        summary_data = []
        
        for expert_num in sorted(expert_data.keys()):
            fc1 = expert_data[expert_num]['fc1'] if expert_data[expert_num]['fc1'] else 0
            gelu = expert_data[expert_num]['gelu'] if expert_data[expert_num]['gelu'] else 0
            fc2 = expert_data[expert_num]['fc2'] if expert_data[expert_num]['fc2'] else 0
            
            # Calculate total compute (fc1 + gelu + fc2)
            total_compute = fc1 + gelu + fc2
            
            row_data = {
                'Expert Number': expert_num,
                'fc1': fc1,
                'gelu': gelu,
                'fc2': fc2,
                'total compute': total_compute
            }
            summary_data.append(row_data)
        
        summary_df = pd.DataFrame(summary_data)
        
        # Save to text file with same name as input but .txt extension
        output_file = os.path.splitext(input_file)[0] + '.txt'
        with open(output_file, 'w') as f:
            f.write(f"PIM Expert Operations - Total Cycles\n")
            f.write(f"Decoder-BS64/{folder_name}/pim/{exp_num}\n")
            f.write(f"Note: 2 activation movements (fc1 and fc2)\n")
            f.write(f"{divider_note}\n\n")
            
            # Write activation movements info
            if activation_movements:
                f.write("--- Activation Movements ---\n")
                for detail in activation_details:
                    f.write(f"{detail}\n")
                f.write(f"Total activation movement cycles: {total_activation_cycles}\n")
                f.write("----------------------------\n\n")
            
            f.write("="*80 + "\n")
            f.write(summary_df.to_string(index=False))
        
        act_info = f" | Act movements: {total_activation_cycles}" if activation_movements else ""
        print(f"    ✓ Processed {len(expert_data)} experts{act_info} → {output_file}")

print(f"\n{'='*80}")
print(f"✓ DECODER-BS64 PIM ANALYSIS COMPLETE!")
print(f"{'='*80}")
print()
print("Summary files (.txt) created for each experiment folder.")
print("Each file contains expert operation timings for all 64 experts.")
print("Activation movements are divided by:")
print("  BS64 → ÷32")
print("  BS32 → ÷64")
print("  BS16 → ÷128")


DECODER-BS64 PIM ANALYSIS


Processing: 1st/pim/

Found 8 SA_stage_E.tsv file(s)

  Processing experiment 2...
    ✓ Processed 51 experts | Act movements: 2242 → Decoder-BS16/1st/pim/2/SA_stage_E.txt
  Processing experiment 3...
    ✓ Processed 51 experts | Act movements: 2242 → Decoder-BS16/1st/pim/3/SA_stage_E.txt
  Processing experiment 4...
    ✓ Processed 51 experts | Act movements: 2242 → Decoder-BS16/1st/pim/4/SA_stage_E.txt
  Processing experiment 5...
    ✓ Processed 51 experts | Act movements: 2242 → Decoder-BS16/1st/pim/5/SA_stage_E.txt
  Processing experiment 6...
    ✓ Processed 52 experts | Act movements: 2242 → Decoder-BS16/1st/pim/6/SA_stage_E.txt
  Processing experiment 7...
    ✓ Processed 48 experts | Act movements: 2242 → Decoder-BS16/1st/pim/7/SA_stage_E.txt
  Processing experiment 8...
    ✓ Processed 53 experts | Act movements: 2242 → Decoder-BS16/1st/pim/8/SA_stage_E.txt
  Processing experiment 9...
    ✓ Processed 49 experts | Act movements: 2242 → Decoder-BS16